# A Model of the Fed's View on Inflation

### Thomas Hasenzagl$^1$, Filippo Pellegrino$^2$, Lucrezia Reichlin$^3$, Giovanni Ricco$^4$

$^1$ University of Minnesota <br>
$^2$ London School of Economics and Political Science <br>
$^3$ London Business School, Now-Casting Economics, and CEPR <br>
$^4$ University of Warwick and OFCE-SciencesPo <br>


#### Notebook description

This notebook generates the charts and tables from the out-of-sample estimation exercise from the paper "A Model of the Fed's View on Inflation". Before running this notebook you have to run `user_main.jl` using `run_type=3`. 


In [ ]:
# ----- Load libraries -----

using CSV, JLD2;
using Dates, DataFrames, Statistics;
include("./code/Metropolis-Within-Gibbs/MetropolisWithinGibbs.jl");
using Main.MetropolisWithinGibbs;
using PlotlyJS, ORCA;
pltjs = PlotlyJS;

In [ ]:
# User input
titles = ["Real GDP", "Employment", "Unemployment rate", "Oil price", "CPI inflation", "Core CPI inflation", "UOM: Expected inflation", "SPF: Expected inflation"];
scales = ["Bil. Chn. 2009\$", "Thousands", "Percent", "\$/Barrel", "Percent", "Percent", "Percent", "Percent"];

In [ ]:
# Load chunk0
res_chunk0 = load("res_oos_chunk0.jld2");

# Load information about the out-of-sample period
data          = res_chunk0["data_full"];
date          = res_chunk0["date"];
MNEMONIC      = res_chunk0["MNEMONIC"];
end_presample = res_chunk0["end_presample"];
end_oos       = res_chunk0["end_oos"];
oos_length    = res_chunk0["oos_length"];

In [ ]:
# Initialise oos_forecast, α_array and σ_array
oos_forecast = Array{Any}(undef, oos_length);
α_array      = Array{Any}(undef, oos_length);
σ_array      = Array{Any}(undef, oos_length);

# Loop over every out-of-sample period
for i=1:oos_length
    
    # Info
    if i == 1 || mod(i, 10) == 0 || i == oos_length
        println("Reading chunk $(i) (out of $(oos_length))");
    end
    
    # Load i-th chunk
    res = jld2open("res_oos_chunk$(i).jld2", "r");
    
    # Update oos_forecast, α_array and σ_array
    oos_forecast[i] = read(res["distr_fcst"]);
    α_array[i] = read(res["distr_α"]);
    σ_array[i] = read(res["σʸ"]);
end

# Forecast horizon
max_h = size(oos_forecast[1])[1];

In [ ]:
# Initialise point forecasts, random walk forecasts
point_forecast = zeros(oos_length-max_h, size(data,2), max_h) |> Array{Union{Missing, Float64}};
rw_forecast    = zeros(oos_length-max_h, size(data,2), max_h) |> Array{Union{Missing, Float64}};
actual         = zeros(oos_length-max_h, size(data,2), max_h) |> Array{Union{Missing, Float64}};

In [ ]:
# Point forecast and random walk
for i=1:oos_length-max_h
    
    # Drift
    d = mean(diff(data[1:end_presample+i-1,:], dims=1), dims=1)[:];

    # Loop over the forecast horizon
    for hz = 1:max_h
        
        # Random walk benchmark
        if hz == 1
            rw_forecast[i, :, hz] = d .+ data[end_presample+i-1, :];
        else
            rw_forecast[i, :, hz] = d .+ rw_forecast[i, :, hz-1]
        end
        
        # Median forecast
        point_forecast[i, :, hz] = median(oos_forecast[i][hz, :, :], dims=2);
        
        # Actual data
        actual[i, :, hz] = data[end_presample+i+hz-1, :];
    end
end

In [ ]:
# Compute RMSFE
tc_rmsfe = sqrt.(dropdims(mean((actual - point_forecast).^2, dims=1), dims=1));
rw_rmsfe = sqrt.(dropdims(mean((actual - rw_forecast).^2, dims=1), dims=1));

# RMSFE DataFrame
df_rmsfe = DataFrame(tc_rmsfe./rw_rmsfe);
rename!(df_rmsfe, Symbol.(["h$(hz)" for hz=1:max_h]))
CSV.write("./csv_output/rmsfe.csv", df_rmsfe);

##### Stability of the common components

In [ ]:
figures = Array{Any}(undef, 3);

c1 = "rgba(0, 48, 158, .75)"; 
c2 = "rgba(255, 0, 0, .75)";

titles_sub = ["Business Cycle", "Energy Price Cycle", "Common Trend"]
scales_sub = ["", "", ""];

for i=1:3
        
    traces1 = Array{Any}(undef, length(α_array));

    for j=1:length(α_array)
        
        if i==1
            αij = median(α_array[j][1,:,:], dims=2);
        elseif i==2
            αij = median(α_array[j][4,:,:], dims=2);
        elseif i==3
            αij = median(α_array[j][6,:,:], dims=2);
        end
        
        traces1[j] = pltjs.scatter(x=date[1:end-max_h], y=αij[1:end-max_h], name="State", mode="lines", line=attr(width=1), showlegend=false);
    end
    
    traces1 = convert(Array{PlotlyJS.GenericTrace{Dict{Symbol,Any}},1}, traces1)

    layout  = pltjs.Layout(;title=titles_sub[i], titlefont=attr(size=12),
                           xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90, zeroline=false),
                           yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title=scales_sub[i]));

    figures[i] = pltjs.plot(traces1, layout);
end

fig = [figures[1]; figures[2]; figures[3]];

# Size
fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 600;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:3
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

savefig(fig, "./img/factor_revisions.pdf", format="pdf");